In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.41


In [288]:
from IPython.display import Audio
sound_file = 'data/Pd1.wav'

In [9]:
from azureml.core import Workspace
ws = Workspace.create(name='jack_workspace',
                      subscription_id='bcfdb149-908f-49ab-8664-97ab3a3db66c',
                      resource_group='parkinsight',
                      create_resource_group=True,
                      location='southcentralus'
                     )

Deploying KeyVault with name jackworkkeyvault2da07317.
Deploying AppInsights with name jackworkinsightsea2899cd.
Deployed AppInsights with name jackworkinsightsea2899cd. Took 4.62 seconds.
Deploying StorageAccount with name jackworkstoragefd1b92b2c.
Deployed KeyVault with name jackworkkeyvault2da07317. Took 20.82 seconds.
Deployed StorageAccount with name jackworkstoragefd1b92b2c. Took 19.96 seconds.
Deploying Workspace with name jack_workspace.
Deployed Workspace with name jack_workspace. Took 57.33 seconds.
Deploying Compute Target with name cpu-cluster
Deploying Compute Target with name gpu-cluster
Deployed Compute Target with name cpu-cluster. Took 72.61 seconds
Deployed Compute Target with name gpu-cluster. Took 72.94 seconds


In [12]:
ws.write_config(file_name="ws_config.json")

In [15]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config(path='.azureml/ws_config.json')
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

jack_workspace	southcentralus	parkinsight	southcentralus


### Model Validation

In [341]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler

In [336]:
import tensorflow as tf
import keras
from tensorflow.keras import layers
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

In [337]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        tf_model = load_model('tf_model.h5')

In [239]:
model = joblib.load('ridge_model.pkl')

In [338]:
# parkinson dataset for training
df = pd.read_csv('parkinsons_updrs.data')
df = df.loc[:, 'total_UPDRS':'HNR']
df = df.drop('NHR', axis=1)
df.shape

(5875, 13)

In [339]:
# splitting data for validation testing
X = df.loc[:,'Jitter(%)':'HNR']
y = df.total_UPDRS
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [345]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [346]:
prediction = tf_model.predict(X_test)
np.mean(np.abs(prediction-y_test.values.reshape(-1)))

10.323478729006228

In [267]:
# os.makedirs('outputs', exist_ok=True)
# # note file saved in the outputs folder is automatically uploaded into experiment record
# joblib.dump(value=model, filename='ridge_regression.pkl')

In [221]:
# from azureml.core.model import Model
# model = Model.register(model_path = "ridge_regression.pkl",
#                        model_name = "ridge_regression.pkl",
#                        tags = {'area': "parkinson", 'type': "regression"},
#                        description = "Ridge regression model to predict UPDRS score",
#                        workspace = ws)

Registering model ridge_regression.pkl


In [310]:
from azureml.core.model import Model
model = Model.register(model_path = "tf_model.h5",
                       model_name = "tf_model.h5",
                       tags = {'area': "parkinson", 'type': "deep learning"},
                       description = "TensorFlow model to predict UPDRS score",
                       workspace = ws)

Registering model tf_model.h5


In [311]:
%%writefile score.py
import tensorflow as tf
import keras
from tensorflow.keras import layers
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
from azureml.core.model import Model
import numpy as np
import json
import ast

def init():
    global model
    model_path = Model.get_model_path('tf_model.h5')

    with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        model = load_model(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data'][0]
        data = ast.literal_eval(data)
        data = np.array(data)
        result = model.predict(data)
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

Overwriting score.py


In [312]:
from azureml.core.conda_dependencies import CondaDependencies 
cd = CondaDependencies.create()
cd.add_conda_package('numpy')
cd.add_conda_package('scikit-learn')
cd.add_tensorflow_conda_package()
cd.add_conda_package('keras')
# myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn'])

cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')

'myenv.yml'

In [313]:
from azureml.core.image import ContainerImage

imgconfig = ContainerImage.image_configuration(execution_script="score.py", 
                                               runtime="python", 
                                               conda_file="myenv.yml")

In [316]:
%%time
from azureml.core.webservice import Webservice

service = Webservice.deploy_from_model(workspace=ws,
                                       name='parkinsight-tf-real-auth',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=imgconfig,
                                       auth_enabled=False)

service.wait_for_deployment(show_output=True)

TypeError: deploy_from_model() got an unexpected keyword argument 'auth_enabled'

In [322]:
# service.get_keys()

In [321]:
service.scoring_uri

'http://88412c43-00e2-4601-adce-4a30de066290.westus.azurecontainer.io/score'

In [287]:
# from azureml.core.image import Image, ContainerImage
# image_config = ContainerImage.image_configuration(runtime= "python",
#                                  execution_script="score.py",
#                                  conda_file="myenv.yml",
#                                  tags = {'area': "parkinson", 'type': "regression"},
#                                  description = "Image with tensorflow regression model")
# image = Image.create(name = "tensorflow2",
#                      # this is the model object 
#                      models = [model],
#                      image_config = image_config, 
#                      workspace = ws)
# image.wait_for_creation(show_output = True)

Creating image
Running.
NotStarted.............................................
Succeeded
Image creation operation finished for image tensorflow2:1, operation "Succeeded"


In [ ]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                          memory_gb = 1, 
                                          tags = {"data": "parkinson", "type": "regression"}, 
                                           description = 'TF regression model')

In [293]:
from azureml.core.webservice import Webservice
service_name = 'aci-parkinsight-real-4'
service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                            image = image,
                                            name = service_name,
                                            workspace = ws)
service.wait_for_deployment(show_output = True)

In [248]:
test_sample = pd.read_csv('test_data.csv')

In [249]:
test_sample = test_sample.drop('Unnamed: 0', axis=1)

In [250]:
test_sample = test_sample.iloc[:, 1:]

In [230]:
import json
test_sample = json.dumps({'data': [
    test_sample.to_json(orient = 'values')
]})
test_sample = bytes(test_sample,encoding = 'utf8')
prediction = service.run(input_data = test_sample)
print(prediction)

[-28.681054240245544, -7.295375837141805, -19.33207615313343, -28.884528635152833, -24.18091816967509, -21.468899601792543, -20.606010176136294, -21.80772401533445, -16.319255014294157, -23.71586162047879, -18.26614673264345, -31.4952379515421, -32.71794334441283, -11.76469782962895, -28.337047066714305, -0.3965905206793181, -10.904355381755629, -15.483870812647297, -15.123996949206571, -12.151844028454576, -19.7918415809227, -27.452280750104677, -29.836275593211802]


In [236]:
service.scoring_uri

'http://070a6b08-9130-4b05-9874-a3d41846cca2.westus.azurecontainer.io/score'

In [315]:
import requests
import json

test_sample = pd.read_csv('test_data.csv')
test_sample = test_sample.drop('Unnamed: 0', axis=1)
test_sample = test_sample.iloc[:, 1:]

headers = {'Content-Type':'application/json'}

if service.auth_enabled:
    headers['Authorization'] = 'Bearer '+service.get_keys()[0]

print(headers)
    
test_sample = json.dumps({'data': [
    test_sample.to_json(orient = 'values')
]})

response = requests.post(service.scoring_uri, data=test_sample, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())

{'Content-Type': 'application/json'}
200
0:00:00.268985
[[-105.87728881835938], [-62.337276458740234], [-85.45500183105469], [-107.1661605834961], [-97.08382415771484], [-91.53104400634766], [-89.83670806884766], [-92.01820373535156], [-81.48261260986328], [-96.17963409423828], [-84.88692474365234], [-111.58415222167969], [-114.44050598144531], [-72.18748474121094], [-105.34400177001953], [-47.38368606567383], [-70.22252655029297], [-79.56731414794922], [-78.93351745605469], [-73.06124877929688], [-88.11766052246094], [-103.48833465576172], [-108.2298583984375]]


In [263]:
service.scoring_uri

'http://070a6b08-9130-4b05-9874-a3d41846cca2.westus.azurecontainer.io/score'